In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

In [2]:
iris = load_iris()
X,y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
joblib.dump(clf, 'models/random_forest.pkl')

['models/random_forest.pkl']

In [4]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import joblib

cls = joblib.load('./models/random_forest.pkl')
initial_type =[('float_input', FloatTensorType([None, 4]))]
onnx_model = convert_sklearn(cls, initial_types=initial_type)

with open('./models/model.onnx' , 'wb') as output:
    output.write(onnx_model.SerializeToString())

In [10]:
import onnxruntime as rt
import numpy as np

data = np.array([[4.5,4.9,5.1,5.4], [4.7,4.1,5.1,5.1], [1.1,1.1,1.3,5.9]])
session = rt.InferenceSession("./models/model.onnx")

ip = session.get_inputs()[0].name
op = session.get_outputs()[0].name

preds = session.run([op], {ip : data.astype(np.float32)})[0]
preds

array([2, 2, 0], dtype=int64)